In [15]:
using Random
using LinearAlgebra
using NBInclude
@nbinclude("deal_features.ipynb")

human_readable (generic function with 1 method)

In [16]:
function generate_cost(hands_NS,random_trial=5;random_cost=false)
    if random_cost
        return rand(52,1)
    end
    costs=zeros(52,1)
    for i in 1:random_trial
        deal=random_deal(hands)
        #---- generate cost vector here
        
        #-----
        costs +=costs
    end
    return costs./random_trial
end
function generate_random_instances(size)
    data=Array{Float64,3}(undef,3,52,size)
    for i in 1:size
        deal=random_deal()
        data[1,:,i]=deal[:,1]
        data[2,:,i]=deal[:,2]
        data[3,:,i]=generate_cost(deal[:,1:2],random_cost=true)
    end
    return data
end

generate_random_instances (generic function with 1 method)

In [17]:
instance_size=1000;
instances = generate_random_instances(instance_size);
turn_depth=6;
costs=[]
iteration_size=1000
for i in 1:iteration_size
    instance= instances[:,:,rand(1:instance_size)]; #choose a random deal
    bidding_hist=zeros(36);
    for t in 1:turn_depth
        if t==1
            last_bid=1
        else
            last_bid=maximum(findall(x-> x==1,bidding_hist)) #the nature of an action
        end
        current_bid=rand(last_bid:36) #randomly bid a contract
        if last_bid == current_bid #if current bid is pass
            break;
        end
        bidding_hist[current_bid]=1;
    end
    
    contract=1
    try
        contract=findall(x-> x==1,bidding_hist)[1]
    catch
        contract=1
    end
    cost=instance[3,contract]   # cost of declaring this contract
    push!(costs,cost)
end

mean(costs)   # overall success of the baseline

0.481380115073463